In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
!pip install openpyxl
from collections import defaultdict
from matplotlib.colors import TwoSlopeNorm
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import random
import matplotlib.patches as mpatches
from matplotlib.cm import get_cmap
import matplotlib.gridspec as gridspec
import seaborn as sns
from matplotlib.colors import LogNorm

# Functions

In [2]:
def apply_stepwise_coverage(loss, thresholds, payout_rates):
    covered = 0
    previous_thresh = 0
    
    for thresh, rate in zip(thresholds, payout_rates):
        if loss <= thresh:
            covered += (loss - previous_thresh) * rate
            break
        else:
            covered += (thresh - previous_thresh) * rate
            previous_thresh = thresh

    return covered

# Parameters

In [3]:
# model specifications
export = True
save = True
plot = False
pls_print = False

In [4]:
input_clustering = '_new' # '_new' or ''
optimal_cluster = 'st_cluster_final_gid' # 'st_cluster_final_gid' or 'st_cluster_3_5_7'

In [5]:
# Scenario
test_case = "random"  #historic, block or random

# Premium Type
premium_type = '' # '_Full' '_Discount' or '' - which is current

# Reinsurance Rules
use_reinsurance = False
base_case = ''
if use_reinsurance == False:
    base_case = '_base_case'

In [6]:
if test_case == 'historic':
    simulations = 1
else:
    steps = 100
    block_size = 10
    simulations = 1000

In [7]:
# Base CPI for adjustment (e.g., 2024)
BASE_CPI = 313.3

# Reinsurance specs
re_thres_vec = [7_000_000_000, 9_000_000_000, 11_000_000_000]
re_payout_vec = [0.120334, 0.258584, 0]
ils_thres_vec = [6_000_000_000, 7_000_000_000, 8_000_000_000, 9_000_000_000, 10_000_000_000, 11_000_000_000]
ils_payout_vec = [0.025, 0.10, 0.2625, 0.35, 0.2375, 0]

# Data Load

## Geospatial

In [8]:
# Load the shapefile for US counties
shapefile_path = '../Local_Data/Geospatial/tl_2019_us_county.shp'
gdf_counties = gpd.read_file(shapefile_path)

# Load the shapefile for US states
state_shapefile_path = '../Local_Data/Geospatial/cb_2018_us_state_20m.shp'
gdf_states = gpd.read_file(state_shapefile_path)

# Load the shapefile for US rivers
rivers_shapefile_path = '../Local_Data/Rivers/rs16my07.shp'
gdf_rivers = gpd.read_file(rivers_shapefile_path)

# Load the shapefile for US urban areas
urban_shapefile_path = '../Local_Data/Geospatial/cb_2018_us_ua10_500k.shp'
gdf_urban = gpd.read_file(urban_shapefile_path)

## NFIP Claims

In [9]:
if input_clustering == '_new':
    clustered_claims = pd.read_csv("../2_Low_Return_Period/new_clusters_9.24.25.csv")
    optimal_cluster = 'st_cluster_final_gid'
else:
    clustered_claims = pd.read_csv("../2_Low_Return_Period/Clusters/2025_all/clustered_claims_sensitivity.csv")
clustered_claims['countyCode']=clustered_claims['countyCode'].apply(lambda x: str(x).zfill(5))
clustered_claims['stateCode']=clustered_claims['countyCode'].str[:2]

/tmp/ipykernel_2052/790541558.py:2: DtypeWarning: Columns (7,51,66) have mixed types. Specify dtype option on import or set low_memory=False.
  clustered_claims = pd.read_csv("../2_Low_Return_Period/new_clusters_9.24.25.csv")


In [10]:
# Create the new field 'percentageDamageAmount'
clustered_claims['percentageBuildingDamageAmount'] = ((clustered_claims['buildingDamageAmount'] / clustered_claims['buildingPropertyValue']) * 100).clip(upper=100)
# Create new field for 'totalClaimPaid'
clustered_claims['totalClaimPaid'] = clustered_claims['amountPaidOnBuildingClaim'].fillna(0) + clustered_claims['amountPaidOnContentsClaim'].fillna(0)
# Filter stateOwnedIndicator is True
clustered_claims = clustered_claims[clustered_claims['stateOwnedIndicator'] != True]

In [11]:
# Load CPI data
CPI_insurance = pd.read_csv("../Local_Data/BLS_Data/US_BLS_CPIAUCSL.csv", parse_dates=["DATE"]) 
#CPI_insurance = pd.read_csv("../Local_Data/BLS_Data/CPI_by_Year_Month.csv") for homeowners insurance CPI

# Ensure 'CPIAUCSL' is numeric
CPI_insurance["CPIAUCSL"] = pd.to_numeric(CPI_insurance["CPIAUCSL"], errors="coerce")

# Handle missing or non-numeric values by forward and backward filling
CPI_insurance["CPIAUCSL"].fillna(method="ffill", inplace=True)
CPI_insurance["CPIAUCSL"].fillna(method="bfill", inplace=True)

# Extract year and calculate annual average CPI
CPI_insurance['Year'] = CPI_insurance["DATE"].dt.year
CPI_insurance = CPI_insurance.groupby('Year')["CPIAUCSL"].mean().reset_index()
CPI_insurance = CPI_insurance.rename(columns={"CPIAUCSL": "CPI"})

# Filter claims to only include those from 1998 to present
clustered_claims['dateOfLoss'] = pd.to_datetime(clustered_claims['dateOfLoss'])

# Extract month (abbreviated) and year from 'dateOfLoss'
clustered_claims.loc[:, 'yearOfLoss'] = clustered_claims['dateOfLoss'].dt.year

# Merge clustered_claims with CPI_insurance on month (abbreviation) and year
claims_merged = pd.merge(clustered_claims, CPI_insurance, left_on=['yearOfLoss'], right_on=['Year'])

# Create 'adjustedClaim' column by dividing the CPI for May 2024 by the CPI at the time of loss, and multiplying by 'totalClaimPaid'
claims_merged['adjustedClaim'] = (BASE_CPI / claims_merged['CPI']) * claims_merged['totalClaimPaid']

# Drop unnecessary columns and keep relevant ones
adjusted_claims = claims_merged[['dateOfLoss', 'totalClaimPaid', 'adjustedClaim', 'yearOfLoss']]

clustered_claims = claims_merged

/tmp/ipykernel_2052/1305496296.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  CPI_insurance["CPIAUCSL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_2052/1305496296.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  CPI_insurance["CPIAUCSL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_2052/1305496296.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an

In [12]:
if input_clustering == '_new':
    pass
else:
    # Katrina
    katrina = 4
    katrina_numbers =[4520, 1707, 1561, 1246, 2187, 1481] 
    
    clustered_claims[optimal_cluster] = clustered_claims[optimal_cluster].apply(
        lambda x: katrina if x in katrina_numbers else x
    )
    
    # Sandy
    sandy = 87
    sandy_numbers =[1603, 1507, 1252] 
    
    clustered_claims[optimal_cluster] = clustered_claims[optimal_cluster].apply(
        lambda x: sandy if x in sandy_numbers else x
    )

    # Harvey
    harvey = 327
    harvey_numbers =[3430, 5719, 5614, 5712, 5678, 5611, 1456, 1736] 
    
    clustered_claims[optimal_cluster] = clustered_claims[optimal_cluster].apply(
        lambda x: harvey if x in harvey_numbers else x
    )
    
    # Ian
    ian = 166
    ian_numbers =[3907, 4050] 
    
    clustered_claims[optimal_cluster] = clustered_claims[optimal_cluster].apply(
        lambda x: ian if x in ian_numbers else x
    )
    
    # ike
    ike = 8
    ike_numbers =[222, 1559, 1419] 
    
    clustered_claims[optimal_cluster] = clustered_claims[optimal_cluster].apply(
        lambda x: ike if x in ike_numbers else x
    )

    # la_2016
    la_2016 = 295
    la_2016_numbers =[3731, 2086, 2053, 3617, 3557] 
    
    clustered_claims[optimal_cluster] = clustered_claims[optimal_cluster].apply(
        lambda x: la_2016 if x in la_2016_numbers else x
    )
    
    # ivan
    ivan = 6
    ivan_numbers =[159, 364, 4003, 4459, 164, 143, 434]
    
    clustered_claims[optimal_cluster] = clustered_claims[optimal_cluster].apply(
        lambda x: ivan if x in ivan_numbers else x
    )
    
    # helene
    helene = 190
    helene_numbers =[195] 
    
    clustered_claims[optimal_cluster] = clustered_claims[optimal_cluster].apply(
        lambda x: helene if x in helene_numbers else x
    )

In [13]:
# Cluster numbers and names
cluster_num = [4, 87, 327, 166, 8, 295, 6, 190]
cluster_names = ["Katrina", "Sandy", "Harvey", "Ian", "Ike", "LA 2016 Floods", "Ivan", "Helene"]

## Nino 3.4

In [14]:
nino34_path = '../Local_Data/Climatological_Index/nino34.long.anom.hadisst.csv'
nino34 = pd.read_csv(
    nino34_path,
    skiprows=1,
    header=None,  # No header in the file
    names=["datetime", "ANOM"],  # Assign column names
    parse_dates=["datetime"]  # Parse first column as datetime
)
# Drop last 12 rows
nino34 = nino34.iloc[:-12]

# Extract year
nino34["YR"] = nino34["datetime"].dt.year

In [15]:
# Ensure ANOM is numeric
nino34['ANOM'] = pd.to_numeric(nino34['ANOM'], errors='coerce')

# Group by year
grouped = nino34.groupby('YR')

# Compute nino and nina
nino = grouped.apply(lambda g: g[g['ANOM'] > 0]['ANOM'].sum())
nina = grouped.apply(lambda g: g[g['ANOM'] < 0]['ANOM'].sum())

# Combine into a new DataFrame
enso_ann_index = pd.DataFrame({
    'YR': nino.index,
    'nino': nino.values,
    'nina': nina.values
}).reset_index(drop=True)

# Normalize nina by its minimum (note: nina is negative, so this preserves relative magnitude)
enso_ann_index['nina_norm'] = enso_ann_index['nina'] / enso_ann_index['nina'].min()

# Normalize nino by its maximum
enso_ann_index['nino_norm'] = enso_ann_index['nino'] / enso_ann_index['nino'].max()

/tmp/ipykernel_2052/3208843501.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nino = grouped.apply(lambda g: g[g['ANOM'] > 0]['ANOM'].sum())
/tmp/ipykernel_2052/3208843501.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nina = grouped.apply(lambda g: g[g['ANOM'] < 0]['ANOM'].sum())


## NFIP RR2 Premiums

In [16]:
if premium_type == '_Full':
    risk_policies = pd.read_excel('../Local_Data/NFIP_Data/updatedNFIPdata_2025.xlsx', sheet_name='2024')
    risk_policies['County Code'] = risk_policies['County Code'].astype(int).astype(str)
    risk_policies['County Code'] = risk_policies['County Code'].apply(lambda x: str(x).zfill(5))
    risk_policies['State'] = risk_policies['State Names'].str.strip().str.upper()
    risk_policies['Policies in Force'] = risk_policies['Policy Count']
    
    aggregated_risk_policies = risk_policies.groupby(['County Code', 'State']).agg({
        'Policies in Force': 'sum',
        'Total Written Premium + FPF': 'sum',
        'Total Annual Payment': 'sum'
    }).reset_index()

elif premium_type == '_Discount':
    risk_policies = pd.read_csv('../Local_Data/NFIP_Data/UpdatedNFIPData_2024_V4.csv')
    risk_policies['countyCode'] = risk_policies['countyCode'].astype(int).astype(str)
    risk_policies['countyCode'] = risk_policies['countyCode'].apply(lambda x: str(x).zfill(5))
    risk_policies['State'] = risk_policies['State Name'].str.strip().str.upper()
    risk_policies['Policies in Force'] = risk_policies['TotalPolicies']
    risk_policies['Total Written Premium + FPF'] = risk_policies['actualPremium'] + risk_policies['federalPolicyFee']
    
    aggregated_risk_policies = risk_policies.groupby(['countyCode', 'State']).agg({
        'Policies in Force': 'sum',
        'Total Written Premium + FPF': 'sum',
        'Total Annual Payment': 'sum'
    }).reset_index()

else:
    risk_policies = pd.read_excel('../Local_Data/NFIP_Data/nfip_policy-information-by-state_20240531.xlsx', sheet_name='PIF')

    risk_policies['County'] = risk_policies['County'].str.strip()
    risk_policies['State'] = risk_policies['State'].str.strip()
    
    aggregated_risk_policies = risk_policies.groupby(['County', 'State']).agg({
        'Policies in Force': 'sum',
        'Total Written Premium + FPF': 'sum',
        'Total Annual Payment': 'sum'
    }).reset_index()

In [17]:
# Manually create the dictionary mapping state FIPS codes to state names
state_fips_dict = {
    "01": "ALABAMA",
    "02": "ALASKA",
    "04": "ARIZONA",
    "05": "ARKANSAS",
    "06": "CALIFORNIA",
    "08": "COLORADO",
    "09": "CONNECTICUT",
    "10": "DELAWARE",
    "11": "DISTRICT OF COLUMBIA",
    "12": "FLORIDA",
    "13": "GEORGIA",
    "15": "HAWAII",
    "16": "IDAHO",
    "17": "ILLINOIS",
    "18": "INDIANA",
    "19": "IOWA",
    "20": "KANSAS",
    "21": "KENTUCKY",
    "22": "LOUISIANA",
    "23": "MAINE",
    "24": "MARYLAND",
    "25": "MASSACHUSETTS",
    "26": "MICHIGAN",
    "27": "MINNESOTA",
    "28": "MISSISSIPPI",
    "29": "MISSOURI",
    "30": "MONTANA",
    "31": "NEBRASKA",
    "32": "NEVADA",
    "33": "NEW HAMPSHIRE",
    "34": "NEW JERSEY",
    "35": "NEW MEXICO",
    "36": "NEW YORK",
    "37": "NORTH CAROLINA",
    "38": "NORTH DAKOTA",
    "39": "OHIO",
    "40": "OKLAHOMA",
    "41": "OREGON",
    "42": "PENNSYLVANIA",
    "44": "RHODE ISLAND",
    "45": "SOUTH CAROLINA",
    "46": "SOUTH DAKOTA",
    "47": "TENNESSEE",
    "48": "TEXAS",
    "49": "UTAH",
    "50": "VERMONT",
    "51": "VIRGINIA",
    "53": "WASHINGTON",
    "54": "WEST VIRGINIA",
    "55": "WISCONSIN",
    "56": "WYOMING"
}

# Convert the dictionary to a DataFrame for easy merging
state_df = pd.DataFrame(list(state_fips_dict.items()), columns=['State FIPS', 'State Name'])

In [18]:
state_abbrev_dict = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
    '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL',
    '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN',
    '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME',
    '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
    '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH',
    '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
    '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
    '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI',
    '56': 'WY'
}

In [19]:
state_claims = pd.read_csv('../Local_Data/NFIP_Data/All_Claims_by_Year.csv')
state_claims = state_claims.groupby("State").aggregate({'Total Claim Dollars Paid':'mean','Total Paid Claims':'mean'}).reset_index()
state_claims['State'] = state_claims['State'].str.upper()
state_claims = state_claims.merge(state_df, left_on='State', right_on='State Name', how='left')
state_policies = aggregated_risk_policies.groupby("State").aggregate({
    'Policies in Force':'sum',
    'Total Written Premium + FPF':'sum',
    'Total Annual Payment':'sum'}).reset_index()
state_merged = state_claims.merge(state_policies, left_on='State', right_on='State', how='left')
state_merged['meanLoss'] = state_merged['Total Written Premium + FPF']-state_merged['Total Claim Dollars Paid']

# Merge the aggregated data with the shapefile
gdf_states = gdf_states.merge(state_merged, left_on='GEOID', right_on='State FIPS', how='left')

In [20]:
# Step 1: Compute total premium
total_premium = gdf_states["Total Written Premium + FPF"].sum()

# Step 2: Create a new column for percentage of total
gdf_states["Contribution %"] = 100 * gdf_states["Total Written Premium + FPF"] / total_premium

# Step 3: Create a new column for value of the pool
gdf_states["Benefit"] = 100 / gdf_states["Contribution %"]
gdf_states["OG_Benefit"] = gdf_states["Benefit"]
og_gain_balance = np.min(gdf_states["Benefit"])

# Step 4: Create a new column for the withdrawal max
gdf_states["Withdrawal_threshold"] = gdf_states["Total Written Premium + FPF"]*og_gain_balance
gdf_states["OG_Withdrawal_threshold"] = gdf_states["Withdrawal_threshold"]

# Remove AL and HI for simulation
gdf_states = gdf_states[~gdf_states['State'].isin(['ALASKA', 'HAWAII'])]

In [21]:
total_premium

np.float64(3873939504.0)

# Main

In [22]:
# Define Hurricanes
if input_clustering == '':
    hurricanes = [4, 87, 327] # Katrina, Sandy, Harvey (only storms above 6b thres)
else:
    hurricanes = [1734, 2214, 2578] # Katrina, Sandy, Harvey
    clustered_claims["year"] = clustered_claims["Year"]

clustered_claims["is_TS"] = clustered_claims[optimal_cluster].isin(hurricanes)

In [23]:
# Initialization
years_arr = np.array(sorted(clustered_claims["dateOfLoss"].dt.year.unique()), dtype=int)
min_year, max_year = years_arr.min(), years_arr.max()

# For block bootstrap, keep the horizon equal to the span you're resampling over
steps = len(years_arr) if test_case == "historic" else steps

# Global containers
cluster_state_output = []     # Cluster-level payouts per state-year
state_balance_output = []     # State-level financial summaries
final_balance_sheets = []     # End-of-simulation state balances

# Initialize transition tracker and sign memory
balance_transitions = []
previous_positive = {}  # Keyed by (simulation, state): True/False

for n in range(simulations):
    print(f'\nSimulation {n + 1}/{simulations}')

    # Year sampling
    if test_case == "random":
        sampled_years = np.random.choice(years_arr, size=steps, replace=True).tolist()
        synthetic_years = list(range(min_year, min_year + steps))

    elif test_case == "block":
        n_blocks = int(np.ceil(steps / block_size))
        
        # Get all years in historical record
        years = sorted(clustered_claims["dateOfLoss"].dt.year.unique())
        min_year, max_year = min(years), max(years)
        
        # All valid block start years
        start_years = list(range(min_year, max_year - block_size + 2))
        
        # Sample block start years with replacement
        sampled_starts = np.random.choice(start_years, size=n_blocks, replace=True)
        
        # Build sampled (historical) years from the blocks
        sampled_years = []
        for start in sampled_starts:
            block = list(range(start, start + block_size))
            sampled_years.extend(block)
        
        # Truncate to exact desired length
        sampled_years = sampled_years[:steps]
        
        # Synthetic timeline (same length as steps, starting from 1978 or other baseline)
        synthetic_years = list(range(min_year, min_year + steps))

    else:
        sampled_years = years_arr.tolist()
        synthetic_years = years_arr.tolist()

    # Initialize balance sheet for this simulation
    balance_sheet = gdf_states.copy()
    balance_sheet["STATEFP"] = balance_sheet["STATEFP"].astype(str)
    balance_sheet["contribution"] = 0
    balance_sheet["nfip_balance"] = 0

    for i in range(steps):
        sample_year = sampled_years[i]
        display_year = synthetic_years[i]

        if pls_print:
            print(f'  Processing year {sample_year}')

        # Subset claims for current year
        year_claims = clustered_claims[clustered_claims["year"] == sample_year]

        # Compute state-cluster claims
        state_cluster_damage = (
            year_claims
            .groupby(["stateCode", optimal_cluster])["adjustedClaim"]
            .sum()
            .reset_index()
        )
        state_cluster_damage["stateCode"] = state_cluster_damage["stateCode"].astype(str)

        # Merge with base balance sheet
        temp = balance_sheet.merge(
            state_cluster_damage,
            how="left",
            left_on="STATEFP",
            right_on="stateCode"
        )
        temp["adjustedClaim"] = temp["adjustedClaim"].fillna(0)

        # Initialize reinsurance and ILS columns
        temp["re_covered"] = 0.0
        temp["ils_covered"] = 0.0
        temp["total_covered"] = 0.0

        if use_reinsurance:
            # Compute total loss per cluster and find high-loss clusters
            cluster_damage = (
                year_claims
                .groupby(optimal_cluster)["adjustedClaim"]
                .sum()
                .reset_index()
                .merge(
                    clustered_claims[[optimal_cluster, "is_TS"]].drop_duplicates(),
                    on=optimal_cluster,
                    how="left"
                )
            )
            min_thresh = min(re_thres_vec[0], ils_thres_vec[0])
            high_loss_clusters = cluster_damage[cluster_damage["adjustedClaim"] > min_thresh]
    
            # Allocate coverage only to affected state-cluster pairs
            for _, row in high_loss_clusters.iterrows():
                cluster = row[optimal_cluster]
                total_loss = row["adjustedClaim"]
                is_ts = row["is_TS"]
    
                # State-wise breakdown of this cluster
                state_breakdown = (
                    year_claims[year_claims[optimal_cluster] == cluster]
                    .groupby("stateCode")["adjustedClaim"]
                    .sum()
                    .reset_index()
                )
                state_breakdown["proportion"] = state_breakdown["adjustedClaim"] / total_loss
    
                re_covered = apply_stepwise_coverage(total_loss, re_thres_vec, re_payout_vec)
                ils_covered = apply_stepwise_coverage(total_loss, ils_thres_vec, ils_payout_vec) if is_ts else 0
                total_cluster_covered = re_covered + ils_covered
    
                # In-place update of temp for each state in the cluster
                for _, s_row in state_breakdown.iterrows():
                    state = s_row["stateCode"]
                    prop = s_row["proportion"]
    
                    mask = (temp["STATEFP"] == state) & (temp[optimal_cluster] == cluster)
                    temp.loc[mask, "re_covered"] += prop * re_covered
                    temp.loc[mask, "ils_covered"] += prop * ils_covered
                    temp.loc[mask, "total_covered"] += prop * total_cluster_covered

        # Annotate metadata
        temp["year"] = display_year
        temp["simulation"] = n

        # Store cluster-level results
        cluster_state_output.append(
            temp[[
                "STATEFP", "stateCode", optimal_cluster, "adjustedClaim",
                "re_covered", "ils_covered", "total_covered",
                "year", "simulation"
            ]]
        )

        # --- State-level financial tracking ---
        state_summary = (
            temp.groupby("STATEFP")[["adjustedClaim", "re_covered", "ils_covered", "total_covered"]]
            .sum()
            .reset_index()
        )

        # Add premiums and calculate contributions
        state_summary = state_summary.merge(
            balance_sheet[["STATEFP", "Total Written Premium + FPF"]],
            on="STATEFP", how="left"
        ).rename(columns={"Total Written Premium + FPF": "premium"}).fillna({"premium": 0})

        state_summary["nfip_payout"] = state_summary["adjustedClaim"] - state_summary["total_covered"]
        state_summary["contribution"] = state_summary["premium"] - state_summary["nfip_payout"]
        state_summary["year"] = display_year
        state_summary["simulation"] = n
        state_summary = state_summary.merge(balance_sheet[["STATEFP", "nfip_balance"]], on="STATEFP", how="left")

        # Merge contributions, ensuring all states retained
        balance_sheet = balance_sheet.merge(
            state_summary[["STATEFP", "contribution"]],
            on="STATEFP", how="left",
            suffixes=('', '_new')
        )
        
        # Safely fill and update contribution column
        if "contribution_new" in balance_sheet.columns:
            balance_sheet["contribution"] = balance_sheet["contribution_new"].fillna(0.0)
            balance_sheet = balance_sheet.drop(columns=["contribution_new"])
        else:
            balance_sheet["contribution"] = 0.0
        
        # Update running balance
        balance_sheet["nfip_balance"] += balance_sheet["contribution"]

        # Track balance transitions from positive to negative
        for _, row in balance_sheet.iterrows():
            state = row["STATEFP"]
            sim_key = (n, state)
            current_balance = row["nfip_balance"]
            was_positive = previous_positive.get(sim_key, True)  # Default to True at start
        
            # Check for transition from positive to negative
            if was_positive and current_balance < 0:
                # Find highest-loss cluster for this state-year
                state_clusters = temp[temp["STATEFP"] == state]
                if not state_clusters.empty:
                    max_row = state_clusters.loc[state_clusters["adjustedClaim"].idxmax()]
                    balance_transitions.append({
                        "simulation": n,
                        "year": display_year,
                        "STATEFP": state,
                        "top_cluster": max_row[optimal_cluster],
                        "cluster_loss": max_row["adjustedClaim"]
                    })
        
            # Update current balance sign
            previous_positive[sim_key] = current_balance >= 0

        # Store state-year summary
        state_balance_output.append(state_summary)

    # Store final balance sheet for this simulation
    balance_sheet["simulation"] = n
    final_balance_sheets.append(balance_sheet.copy())


Simulation 1/1000

Simulation 2/1000

Simulation 3/1000

Simulation 4/1000

Simulation 5/1000

Simulation 6/1000

Simulation 7/1000

Simulation 8/1000

Simulation 9/1000

Simulation 10/1000

Simulation 11/1000

Simulation 12/1000

Simulation 13/1000

Simulation 14/1000

Simulation 15/1000

Simulation 16/1000

Simulation 17/1000

Simulation 18/1000

Simulation 19/1000

Simulation 20/1000

Simulation 21/1000

Simulation 22/1000

Simulation 23/1000

Simulation 24/1000

Simulation 25/1000

Simulation 26/1000

Simulation 27/1000

Simulation 28/1000

Simulation 29/1000

Simulation 30/1000

Simulation 31/1000

Simulation 32/1000

Simulation 33/1000

Simulation 34/1000

Simulation 35/1000

Simulation 36/1000

Simulation 37/1000

Simulation 38/1000

Simulation 39/1000

Simulation 40/1000

Simulation 41/1000

Simulation 42/1000

Simulation 43/1000

Simulation 44/1000

Simulation 45/1000

Simulation 46/1000

Simulation 47/1000

Simulation 48/1000

Simulation 49/1000

Simulation 50/1000

Simulati

In [24]:
# Compile all outputs
cluster_state_df = pd.concat(cluster_state_output, ignore_index=True)
state_balance_df = pd.concat(state_balance_output, ignore_index=True)
final_balances_df = pd.concat(final_balance_sheets, ignore_index=True)
balance_transition_df = pd.DataFrame(balance_transitions)

if export:
    cluster_state_df.to_csv(f'Results/cluster_state_{test_case}{input_clustering}{premium_type}{base_case}.csv')
    state_balance_df.to_csv(f'Results/state_balance_{test_case}{input_clustering}{premium_type}{base_case}.csv')
    final_balances_df.to_csv(f'Results/final_balances_{test_case}{input_clustering}{premium_type}{base_case}.csv')
    balance_transition_df.to_csv(f'Results/balance_transition_{test_case}{input_clustering}{premium_type}{base_case}.csv')